## Abrindo o arquivo

In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
import random

In [2]:
treinamento = pd.read_csv("/content/drive/MyDrive/Arquivos Colab/Titanic/train.csv")
treinamento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Removendo o PassengerID

In [3]:
treinamento = treinamento.drop(['PassengerId', 'Cabin'], axis=1)

## Valores faltando

Fare e Embarked

In [4]:
l = ['C', 'S', 'Q']
treinamento['Embarked'].fillna(random.choice(l), inplace=True)
treinamento['Fare'].fillna(treinamento['Fare'].median())
treinamento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB


Age

In [5]:
# Preenche a vaga faltando em age com a média:
treinamento['Age'].fillna(treinamento['Age'].median(), inplace=True)

In [6]:
treinamento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       891 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB


## Lidando com as variáveis que tem um grande número de categorias:

Ticket: Separar os tickets por tipos 

In [14]:
import string
def sep_ticket(data_ticket):
    ticket_type = []

    for i in range(len(data_ticket)):

            ticket =data_ticket.iloc[i]

            for c in string.punctuation:
                ticket = ticket.replace(c,"")
                splited_ticket = ticket.split(" ")   
            if len(splited_ticket) == 1:
                ticket_type.append('NA')
            else: 
                ticket_type.append(splited_ticket[0])
    return ticket_type 

In [8]:
treinamento["ticket_type"] = sep_ticket(treinamento['Ticket'])
treinamento["ticket_type"].value_counts()

NA         665
PC          60
CA          41
A5          21
SOTONOQ     15
STONO       12
WC          10
SCPARIS      7
A4           7
STONO2       6
SOC          6
C            5
FCC          5
SCParis      4
SOPP         3
PP           3
WEP          3
SCAH         3
PPP          2
SWPP         2
SOTONO2      2
Fa           1
SP           1
SOP          1
CASOTON      1
SCA4         1
FC           1
SC           1
SCOW         1
AS           1
Name: ticket_type, dtype: int64

Diminuindo a quantidade de categorias

In [10]:
for t in treinamento['ticket_type'].unique():
    if len(treinamento[treinamento['ticket_type']==t]) < 15:
        treinamento.loc[treinamento.ticket_type ==t, 'ticket_type'] = 'OTHER_T'

In [11]:
treinamento['ticket_type'].value_counts()

NA         665
OTHER_T     89
PC          60
CA          41
A5          21
SOTONOQ     15
Name: ticket_type, dtype: int64

In [12]:
treinamento = treinamento.drop('Ticket', axis=1)

Categorizando e diminuindo a cardinalidade dos nomes

In [13]:
def sep_name(data):
    families=[]
    titles = []
    new_name = []
    for i in range(len(data)):
        name = data.iloc[i]
        if '(' in name:
            name_no_bracket = name.split('(')[0] 
        else:
            name_no_bracket = name
            
        family = name_no_bracket.split(",")[0]
        title = name_no_bracket.split(",")[1].strip().split(" ")[0]
             
        for c in string.punctuation:
            name = name.replace(c,"").strip()
            family = family.replace(c,"").strip()
            title = title.replace(c,"").strip()
            
        families.append(family)
        titles.append(title)
        new_name.append(name)
            
    return families, titles, new_name 

In [15]:
treinamento['family'], treinamento['title'], treinamento['Name']  = sep_name(treinamento.Name)
treinamento = treinamento.drop('Name', axis=1)
treinamento.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,ticket_type,family,title
0,0,3,male,22.0,1,0,7.2500,S,A5,Braund,Mr
1,1,1,female,38.0,1,0,71.2833,C,PC,Cumings,Mrs


Diminuindo a quantidade de categorias

In [16]:
for t in treinamento['title'].unique():
    if len(treinamento[treinamento['title']==t]) <= 15:
        treinamento.loc[treinamento.title ==t, 'title'] = 'OTHER'

Separando x e y

In [17]:
treinamento.head(1)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,ticket_type,family,title
0,0,3,male,22.0,1,0,7.25,S,A5,Braund,Mr


In [20]:
x = treinamento.iloc[:, 1:11]
y = treinamento['Survived']

## Onehot encoding

In [23]:
x_onehot = x

In [24]:
colunas = [ 'title', 'Embarked', 'ticket_type']

for c in colunas:
    x_onehot = pd.concat([x_onehot, pd.get_dummies(x_onehot[c], drop_first = False)], axis =1)

In [25]:
x_onehot = x_onehot.drop(['Embarked', 'ticket_type', 'title'], axis=1)

In [26]:
from sklearn.preprocessing import LabelEncoder
# Faz o encoder:
encoder_sex = LabelEncoder()
encoder_family = LabelEncoder()
x_onehot['Sex'] = encoder_sex.fit_transform(x_onehot['Sex'])
x_onehot['family'] = encoder_family.fit_transform(x_onehot['family'])

In [27]:
x_onehot.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,family,Master,Miss,Mr,Mrs,OTHER,C,Q,S,A5,CA,NA,OTHER_T,PC,SOTONOQ
0,3,1,22.0,1,0,7.2500,73,0,0,1,0,0,0,0,1,1,0,0,0,0,0
1,1,0,38.0,1,0,71.2833,136,0,0,0,1,0,1,0,0,0,0,0,0,1,0


A classe a seguir contem os processos acima:

In [28]:
class LimpaTudo:
  def __init__(self, df):
    self.df = df
  def run(self):
    self._missing()
    self._ticket()
    self._name()
    x = self._onehot()
    d = self._labelen(x)
    return d
  def _missing(self):
    self.df['Age'].fillna(self.df['Age'].median(), inplace=True)
    self.df = self.df.drop('Cabin', axis=1)
    self.df['Fare'].fillna(self.df['Fare'].median(), inplace=True)
  def _ticket(self):
    self.df['ticket_type']=sep_ticket(self.df['Ticket'])
    for t in self.df['ticket_type'].unique():
      if len(self.df[self.df['ticket_type']==t]) < 15:
        self.df.loc[self.df.ticket_type ==t, 'ticket_type'] = 'OTHER_T'
    self.df = self.df.drop('Ticket', axis=1)
  def _name(self):
    self.df['family'], self.df['title'], self.df['Name']  = sep_name(self.df.Name)
    self.df = self.df.drop('Name', axis=1)
    for t in self.df['title'].unique():
      if len(self.df[self.df['title']==t]) <= 15:
        self.df.loc[self.df.title ==t, 'title'] = 'OTHER'
  def _onehot(self):
    df_onehot = self.df
    columns = [ 'title', 'Embarked', 'ticket_type']
    for col in columns:
      df_onehot = pd.concat([df_onehot, pd.get_dummies(df_onehot[col])], axis =1)
    df_onehot = df_onehot.drop(['Embarked', 'ticket_type', 'title'], axis=1)
    return df_onehot
  def _labelen(self, df_onehot):
    encoder_sex = LabelEncoder()
    encoder_family = LabelEncoder()
    df_onehot['Sex'] = encoder_sex.fit_transform(df_onehot['Sex'])
    df_onehot['family'] = encoder_family.fit_transform(df_onehot['family'])
    return df_onehot